# Comparação entre modelos para definir publicação ou não em produção
## Comparar:
* Accuracy geral
* Recall em vazios
* Recall em não vazios
* Velocidade
* Uso de memória

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
SIZE = (224, 224)


# Carregar imagens

In [2]:
caminho_basest = os.path.join('..', 'bases', 'vazios')
caminho_traint = os.path.join(caminho_basest, 'train')
caminho_testt = os.path.join(caminho_basest, 'test')

In [3]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    caminho_testt,
    target_size=SIZE,
    batch_size=128,
    class_mode='binary',
    shuffle=False)

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    caminho_traint,
    target_size=SIZE,
    batch_size=128,
    class_mode='binary',
    shuffle=False)

Found 2317 images belonging to 2 classes.
Found 20845 images belonging to 2 classes.


# Carregar modelo scvvazios.pkl

In [9]:
import numpy as np
from PIL import Image
from sklearn.externals import joblib

SIZE = 128, 128
MODEL_FILE = 'vaziossvc.pkl'


class VazioSVMModel():
    def __init__(self, size=SIZE):
        self.size = size
        self.model = joblib.load(MODEL_FILE)
        self.input_shape = [SIZE[0] * SIZE[1]]

    def image_prepare(self, image: Image):
        image = image.resize(self.size, Image.ANTIALIAS)
        image_array = np.asarray(image).astype('float32')
        # del image
        image_array = image_array[:, :, 0] / 255
        # print(image_array.shape)
        image_array = np.reshape(image_array,
                                 image_array.shape[0] * image_array.shape[1])
        return image_array

    def prepara(self, image):
        return self.image_prepare(image)

    def predict(self, image: Image):
        # O modelo SVM foi treinado em classificação binária
        # 0 para vazio e 1 para não vazio
        y = self.model.predict([self.image_prepare(image)]).tolist()
        return 1 if y[0] == 0. else 0
        # vazio = [{'vazio': y_ == 0} for y_ in y]
        # return vazio
    

model1 = VazioSVMModel()


In [21]:
y_pred = []
for filename in validation_generator.filenames:
    pil_image = Image.open(os.path.join(caminho_testt, filename))
    y_pred.append(model1.predict(pil_image))

In [22]:
y_pred[:20]

[0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1]

In [23]:
validation_generator.labels[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [24]:
sum(y_pred == validation_generator.labels) / len(y_pred)

0.9555459646094088

In [25]:
print(classification_report(validation_generator.labels, y_pred))

             precision    recall  f1-score   support

          0       1.00      0.91      0.95      1166
          1       0.92      1.00      0.96      1151

avg / total       0.96      0.96      0.96      2317



In [26]:
print(confusion_matrix(validation_generator.labels, y_pred))

[[1063  103]
 [   0 1151]]


# Carregar melhor modelo do notebook 01b3

In [58]:
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3, 3),
                         padding='same',
                         activation='relu',
                         input_shape=(*SIZE, 3)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(32, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),
  Conv2D(64, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Conv2D(256, (3, 3), activation='relu'),
  Flatten(),
  Dense(256, activation='relu'),
  Dropout(0.4),
  Dense(1, activation='sigmoid')
 
])


In [59]:
model2.load_weights(os.path.join('..', 'models', 'B3modelweights.17-0.08.hdf5'))
y_pred2 = model2.predict(validation_generator)

In [60]:
y_pred2_labels = y_pred2.reshape(-1) > 0.5

sum(y_pred2_labels == validation_generator.labels) / len(y_pred2_labels)

0.9620198532585239

In [67]:
print(classification_report(validation_generator.labels, y_pred2_labels))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1166
           1       0.93      1.00      0.96      1151

    accuracy                           0.96      2317
   macro avg       0.96      0.96      0.96      2317
weighted avg       0.96      0.96      0.96      2317



In [68]:
print(confusion_matrix(validation_generator.labels, y_pred2_labels))

[[1078   88]
 [   0 1151]]


# Carregar melhor modelo do notebook 02c3

In [49]:
from tensorflow.keras.applications.densenet import DenseNet121

base_model = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='max')

def extract_features(generator, model):
    generator.reset()
    n_images = len(generator.filenames)
    batch_size = generator.batch_size
    m = batch_size *  (n_images // batch_size)  # Arredondar para não ficar espaço vazio
    n = model.output.shape[1]
    features = np.zeros((m, n), np.float32)
    y_ = np.zeros((m, 1), np.float32)
    print(n_images, m, batch_size, m // batch_size)
    for ind in range(m // batch_size):
        batch, y_batch = next(generator)
        features_batch = base_model.predict(batch)
        features[ind * batch_size: (ind * batch_size) + batch_size, :] = features_batch
        y_[ind * batch_size: (ind * batch_size) + batch_size, 0] = y_batch
    print('last batch setted elements %s:%s' % ((ind * batch_size), ((ind * batch_size) + batch_size)))
    return features, y_

features_test, y_test = extract_features(validation_generator, base_model)

2317 2304 128 18
last batch setted elements 2176:2304


In [50]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [51]:
MODEL_DIR = '../models_featureextraction'

epoch = 13
val_loss = 0.10
model.load_weights(
    os.path.join(MODEL_DIR,
                 'Transfermodelweights02c3-classweights.{:02d}-{:.2f}.hdf5'.format(epoch, val_loss)
                ))


In [55]:
y_pred3 = model.predict(features_test)

In [56]:
y_pred3.shape

(2304, 1)

In [57]:
y_pred3_labels = y_pred3.reshape(-1) > 0.5

sum(y_pred3_labels == validation_generator.labels[:len(y_pred3)]) / len(y_pred3)

0.9526909722222222

In [64]:
print(classification_report(validation_generator.labels[:len(y_pred3)], y_pred3_labels))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95      1166
           1       0.94      0.97      0.95      1138

    accuracy                           0.95      2304
   macro avg       0.95      0.95      0.95      2304
weighted avg       0.95      0.95      0.95      2304



In [65]:
print(confusion_matrix(validation_generator.labels[:len(y_pred3)], y_pred3_labels))

[[1096   70]
 [  39 1099]]
